# A clamped-free bar excited by end load

Reference [1].

[1] Malakiyeh, Mohammad Mahdi, Saeed Shojaee, and Klaus-Jürgen Bathe. "The Bathe time integration method revisited for prescribing desired numerical dissipation." Computers & Structures 212 (2019): 289-298.

In [1]:
using LinearAlgebra

In [2]:
function clamped_bar(; N=1000, E=30e6, ρ=7.3e-4, A=1, L=200)
    ℓ = L / N
    K = (E * A / ℓ) * SymTridiagonal(fill(2.0, N), fill(-1.0, N))
    K[end, end] = E * A / ℓ

    M = (ρ * A * ℓ / 2) * Diagonal(vcat(fill(2.0, N-1), 1.0))
    M[end, end] = ρ * A * ℓ / 2

    return M, K
end

clamped_bar (generic function with 1 method)

In [3]:
M, K = clamped_bar(N=5)
M

5×5 Diagonal{Float64,Array{Float64,1}}:
 0.0292   ⋅       ⋅       ⋅       ⋅ 
  ⋅      0.0292   ⋅       ⋅       ⋅ 
  ⋅       ⋅      0.0292   ⋅       ⋅ 
  ⋅       ⋅       ⋅      0.0292   ⋅ 
  ⋅       ⋅       ⋅       ⋅      0.0146

In [4]:
K

5×5 SymTridiagonal{Float64,Array{Float64,1}}:
       1.5e6  -750000.0           ⋅            ⋅            ⋅ 
 -750000.0          1.5e6  -750000.0           ⋅            ⋅ 
        ⋅     -750000.0          1.5e6  -750000.0           ⋅ 
        ⋅            ⋅     -750000.0          1.5e6  -750000.0
        ⋅            ⋅            ⋅     -750000.0     750000.0

## Problem definition

In [5]:
using ReachabilityAnalysis, Plots
using ReachabilityAnalysis: normalize
using LazySets.Approximations: SingleEntryVector
using ExponentialUtilities

using BenchmarkTools

const IA = IntervalArithmetic
const RA = ReachabilityAnalysis

LazySets.set_ztol(Float64, 1e-16)

┌ Info: Precompiling ReachabilityAnalysis [1e97bd63-91d1-579d-8e8d-501d2b57c93f]
└ @ Base loading.jl:1278


1.0e-16

In [6]:
function clamped_free(; N)
    M, K = clamped_bar(N=N)
    C = zeros(N, N) # no damping
    sys = SecondOrderLinearContinuousSystem(M, C, K)
end

function clamped_forced(; N, F=10e3, E=30e6, A=1)
    M, K = clamped_bar(N=N)
    C = zeros(N, N) # no damping
    
    b = vcat(zeros(N-1), F) # the right-most node is excited

    # see Mechanical Vibrations, Gerardin et al, page 250-251 < Force relates to velocity initial cond
    # Fext = vcat(zeros(N), vcat(zeros(N-1), F / (E * A))) ??

    # b = zeros(N)

    sys = SecondOrderAffineContinuousSystem(M, C, K, b)
end

function edirs(i, N)
    ei = zeros(2N)
    ei[i] = 1.0
    return ei, -ei
end

function edirs_single(i, N)
    e1s = SingleEntryVector(i, 2N, 1.0)
    e2s = SingleEntryVector(i, 2N, -1.0)
    return e1s, e2s
end

edirs_single (generic function with 1 method)

## Step response with zero init cond solved with Krylov (NO BLOATING)

In general:

$X_{k+1} = \Phi X_k \oplus V_k$ for $k \geq 1$, where $V_k := \Phi_1(A, \delta)U_k$, and $\Phi := e^{A\delta}$

---

Assume:

$X_0 = 0$, $V_1 \neq 0$, $V_k = 0$ for $k \geq 2$.

Then:

- $X_0 = 0$,
- $X_1 = V_1$,
- $X_2 = \Phi V_1$,
- $X_3 = \Phi^2 V_1$,
- $\vdots$

Given a direction $\ell \in \mathbb{R}^{2N}$, we compute the sequence:

$$
\rho(\ell, V_1), \rho(\ell, \Phi V_1), \rho(\ell, \Phi^2 V_1), \ldots,  \rho(\ell, \Phi^K V_1).
$$

In [7]:
# definitions
δ = 1e-8

# case with forcing
N = 1000
sysf = clamped_forced(N=N)
@time sysnormf = normalize(sysf)
A = state_matrix(sysnormf)
b = sysnormf.U.U.element
Aᵀδ = A' * δ

# discretization of the input
@time φ₁ = RA.Φ₁(A, δ, Val(:base))

V = φ₁ * b |> Singleton;

  2.656259 seconds (6.67 M allocations: 405.591 MiB, 5.70% gc time)
 20.158877 seconds (4.58 M allocations: 5.611 GiB, 3.60% gc time)


In [8]:
#maximum(V.element) # 701289.9    using δ = 1e-7
#maximum(V.element) # 7036.7      using δ = 1e-8
#maximum(V.element) # 70.3        using δ = 1e-9
#maximum(V.element)

In [9]:
NSTEPS = 36_000 * 5
N = 1000

k = 500 # displacement node 500
#k = 700 # displacement node 700
#k = 1500 # velocity node 500

e1, e2 = edirs(k, N)

# see Mechanical Vibrations, Gerardin et al, page 250-251 < Force relates to velocity initial cond
Fext = 10e3
N=1000; E=30e6; A=1; L=200;
L0 = L / N
#v0 = Fext / (E * A) * L0
#X0 = vcat(vcat(zeros(N-1), v0), zeros(N))  |> Singleton
#X0.element[1000]
X0 = Singleton(zeros(2N));

# @time Φ = exp(Matrix(Aᵀδ));
# ρ(Φ * e1, V)

@show δ  * NSTEPS

δ * NSTEPS = 0.0018


0.0018

In [10]:
#@time out1K = forced_nobloating_krylov_2(X0, V, Afᵀδ, e1, NSTEPS; hermitian=false, m=min(30, size(Afᵀδ, 1)), tol=1e-10);
#@time out2K = forced_nobloating_krylov_2(X0, V, Afᵀδ, e2, NSTEPS; hermitian=false, m=min(30, size(Afᵀδ, 1)), tol=1e-10);

out1K = Vector{Float64}(undef, NSTEPS)
out2K = Vector{Float64}(undef, NSTEPS)

@time RA.reach_homog_krylov_LGG09!(out1K, V, Aᵀδ, e1, NSTEPS)
@time RA.reach_homog_krylov_LGG09!(out2K, V, Aᵀδ, e2, NSTEPS);

  4.152240 seconds (15.85 M allocations: 813.533 MiB)
  1.772362 seconds (9.72 M allocations: 519.600 MiB, 10.02% gc time)


In [11]:
@show minimum(out1K)
@show maximum(out1K)

@show minimum(out2K)
@show maximum(out2K)

minimum(out1K) = 0.0
maximum(out1K) = 0.0
minimum(out2K) = 0.0
maximum(out2K) = 0.0


0.0

In [ ]:
@time intervalos = [ReachSet(Interval(-out2K[k+1], out1K[k+1]), IA.Interval(k*δ, (k+1)*δ)) for k in 0:NSTEPS-1]

fp = Flowpipe(intervalos);
@show numrsets(fp)

plot(fp[1:1:1000], vars=(0, 1), xlab="time", ylab="displacement $k", lab="", lw=0.1)

## Step response with zero init cond solved with Krylov (WITH BLOATING)

In [48]:
# definitions
δ = 1e-7

# case with forcing
N = 1000
sysf = clamped_forced(N=N)
@time sysnormf = normalize(sysf)
A = state_matrix(sysnormf)
b = sysnormf.U.U.element
U0 = Singleton(b)
Aᵀδ = A' * δ

A_abs = RA._elementwise_abs(A)

@time Phi2A_abs = RA.Φ₂(A_abs, δ, Val(:base));

@time Eψ0 = RA.sih(Phi2A_abs * RA.sih(A * U0, Val(:concrete)), Val(:concrete));

X0 = (δ * U0) ⊕ Eψ0
V0 = (δ * U0) ⊕ Eψ0;

  0.064054 seconds (101 allocations: 69.086 MiB)
 23.740316 seconds (147 allocations: 5.395 GiB, 4.43% gc time)
  0.381173 seconds (32.01 k allocations: 124.940 MiB, 47.93% gc time)


In [54]:
NSTEPS = 40_000

k = 1000 # displacement node 500
#k = 700 # displacement node 700
#k = 1500 # velocity node 500

e1, e2 = edirs(k, N)

@show δ  * NSTEPS

out1K = Vector{Float64}(undef, NSTEPS)
out2K = Vector{Float64}(undef, NSTEPS)

Vk = vcat(V0, fill(ZeroSet(2N), NSTEPS-1))
@time RA.reach_inhomog_krylov_LGG09!(out1K, X0, Vk, Aᵀδ, e1, NSTEPS)
@time RA.reach_inhomog_krylov_LGG09!(out2K, X0, Vk, Aᵀδ, e2, NSTEPS);

δ * NSTEPS = 0.004
131.647376 seconds (1.60 G allocations: 27.328 GiB, 4.76% gc time)
126.073459 seconds (1.60 G allocations: 27.328 GiB, 4.82% gc time)


In [55]:
@show minimum(out1K)
@show maximum(out1K)

@show minimum(out2K)
@show maximum(out2K)

minimum(out1K) = -3.359280558352365e-5
maximum(out1K) = 3.351011071626167e-5
minimum(out2K) = -3.3025306718204705e-5
maximum(out2K) = 3.408564429058464e-5


3.408564429058464e-5

In [ ]:
@time intervalos = [ReachSet(Interval(-out2K[k+1], out1K[k+1]), IA.Interval(k*δ, (k+1)*δ)) for k in 0:NSTEPS-1]

fp = Flowpipe(intervalos);
@show numrsets(fp)

plot(fp[1:1:1000], vars=(0, 1), xlab="time", ylab="displacement $k", lab="", lw=0.1)

## Free case, no bloating, without Krylov

In [ ]:
# number of elements
N = 1000
sys = clamped_free(N=N)
sysf = clamped_forced(N=N)

# box initial conditions
#X0 = BallInf(ones(2N), 0.08);
X0 = Hyperrectangle(low=vcat(ones(N) .- 0.1, zeros(N)),
                    high=vcat(ones(N) .+ 0.1, zeros(N)))

X0f = Singleton(zeros(2N)); # origin

# step size
δ = 1e-7;

# directions of interest
k = 700 # displacement at node 700
# kv = 1400 # velocity at node 700

dirs = edirs(k, N)
e1 = dirs.directions[1]
e2 = dirs.directions[2]

dirs_single = edirs_single(k, N)
e1s = dirs_single.directions[1]
e2s = dirs_single.directions[2];

$X_{k+1} = \Phi X_k$ for $k \geq 0$ and since we dont consider bloating, $\Phi = e^{A\delta}$ and $X_0$ is the given `X0`.

In [ ]:
# definitions
δ = 1e-7;
@time sysnorm = normalize(sys)
A = state_matrix(sysnorm);
Aᵀ = copy(transpose(A));
Aδ = A*δ
Aᵀδ = Aᵀ*δ

# state evolution matrix
@time Φ = exp(Matrix(A) * δ);
@time Φᵀ = exp(Matrix(Aᵀ) * δ);

# case with forcing
@time sysnormf = normalize(sysf)
Af = state_matrix(sysnormf);
Afᵀ = copy(transpose(Af));
Afδ = Af*δ
Afᵀδ = Afᵀ*δ;
Φf = exp(Matrix(Af) * δ);
Φfᵀ = exp(Matrix(Afᵀ) * δ);

Hence, $X_{k+1} = \Phi^k X_0$ for $k \geq 1$. We are interested in computing the sequence

$$
\rho(\ell, X_0), \rho(\ell, \Phi X_0), \rho(\ell, \Phi^2 X_0), \ldots,  \rho(\ell, \Phi^K X_0).
$$
where $\ell \in \mathbb{R}^{2N}$ is a given direction.

Recall that

$$
\rho(\ell, X_0), \rho((\Phi^T) \ell,  X_0), \rho((\Phi^T)^2\ell,  X_0), \ldots,  \rho((\Phi^T)^K\ell,  X_0).
$$
where $\ell \in \mathbb{R}^{2N}$ is a given direction.

In [ ]:
T = 0.018

T / δ # number of computation steps required

In [ ]:
#spy(Φ)

In [ ]:
@btime ρ($e1, $X0); # 400ns

In [ ]:
@btime ρ($e1s, $X0); # 4ns

In [ ]:
@btime ρ($e1, $Φ * $X0); # 800us

In [ ]:
@btime ρ($e1s, $Φ * $X0); # 10us

In [ ]:
@btime $Φ * $Φ; # 165ms (DENSE)

In [ ]:
Φ² = Φ * Φ;

In [ ]:
165e-3 * 1e6 / 60 / 60 # 45.8 hours

In [ ]:
@btime $A + $A; # 20us

In [ ]:
20e-6 * 1e6 / 60  # 0.2 minutes

In [ ]:
@btime $Aᵀ + $Aᵀ; # 

In [ ]:
@btime $Φᵀ * $e1s; # 2.536 μs

In [ ]:
@btime ρ($Φᵀ * $e1s, $X0); # 3.252 μs

In [ ]:
# ------------------------------------------------------------
# TODO : compare representing Φᵀ sparse vs dense
# TODO : profile  
# ------------------------------------------------------------
function free_nobloating(X₀, Φᵀ, ℓ, NSTEPS)
    rᵢ = copy(ℓ)
    rᵢ₊₁ = similar(rᵢ)

    out = Vector{Float64}(undef, NSTEPS)

    @inbounds for i in 1:NSTEPS
        out[i] = ρ(rᵢ, X₀)
        mul!(rᵢ₊₁, Φᵀ, rᵢ)
        copy!(rᵢ, rᵢ₊₁)
    end
    return out
end

In [ ]:
@btime free_nobloating($X0, $Φᵀ, $e1, 1000);

In [ ]:
1.321 / 1000  # ~ 1 ms per iteration

In [ ]:
@btime free_nobloating($X0, $Φᵀ, $e1s, 1000);

In [ ]:
GC.gc()

In [ ]:
180_000 / 5

In [ ]:
NSTEPS = 36000

In [ ]:
@time out1 = free_nobloating(X0, Φᵀ, e1s, NSTEPS);
@time out2 = free_nobloating(X0, Φᵀ, e2s, NSTEPS);

In [ ]:
length(out1)

In [ ]:
minimum(out1)

In [ ]:
maximum(out1)

In [ ]:
plot([k*δ for k in 0:NSTEPS-1], out1[1:1:NSTEPS], xlab="time", ylab="displacement $k", lab="", c=:blue)

In [ ]:
plot([k*δ for k in 0:NSTEPS-1], out2[1:1:NSTEPS], xlab="time", ylab="displacement $k", lab="", c=:red)

In [ ]:
@time intervalos = [ReachSet(Interval(-out2[k+1], out1[k+1]),
                             IA.Interval(k*δ, (k+1)*δ)) for k in 0:NSTEPS-1]
fp = Flowpipe(intervalos);

In [ ]:
plot(fp(0 .. 0.00025), vars=(0, 1), xlab="time", ylab="displacement $k", lab="", lw=0.1)

## Free case, no bloating, with Krylov

In [ ]:
using ExponentialUtilities

In [ ]:
# this function computes the sequence
# ``ρ(ℓ, Ω₀)``, ``ρ(exp(Aᵀ) * ℓ, Ω₀)``, ``ρ(exp(2Aᵀ) * ℓ, Ω₀)`` until ``ρ(exp(NSTEPS * Aᵀ) * ℓ, Ω₀)``
# using a precomputed Krylov subspace of Aᵀ of order m
function free_nobloating_krylov(X₀, Aᵀ, ℓ, NSTEPS;
                                hermitian=false, m=min(30, size(Aᵀ, 1)), tol=1e-7)

    out = Vector{Float64}(undef, NSTEPS)

    TA, Tb = eltype(Aᵀ), eltype(ℓ)
    T = promote_type(TA, Tb)
    Ks = KrylovSubspace{T, real(T)}(length(ℓ), m)
    arnoldi!(Ks, Aᵀ, ℓ; m=m, ishermitian=hermitian, tol=tol)

    rᵢ = deepcopy(ℓ)

    @inbounds for i in 1:NSTEPS
        out[i] = ρ(rᵢ, X₀)

        # update cache for the next iteration
        expv!(rᵢ, i*1.0, Ks)
    end
    return out
end

Is $A$ hermitian? 

In [ ]:
opnorm(Matrix(Aᵀδ) - Matrix(Aδ), Inf) # NO !

In [ ]:
NSTEPS = 100
ℓ = e1 
#
# NOTE e1s setindex! not defined for SingleEntryVector{Float64}
#
@time outK = free_nobloating_krylov(X0, Aᵀδ, ℓ, NSTEPS; hermitian=false, m=min(30, size(Aᵀδ, 1)), tol=1e-10);

In [ ]:
@time out = free_nobloating(X0, Φᵀ, e1, 100);

In [ ]:
norm(outK - out)

In [ ]:
#=
X0 = Hyperrectangle(low=vcat(ones(N) .- 0.1, zeros(N)),
                    high=vcat(ones(N) .+ 0.1, zeros(N)))
=#

# definitions
δ = 1e-6;

@time sysnorm = normalize(sys)
A = state_matrix(sysnorm);
Aᵀ = copy(transpose(A));
Aδ = A*δ
Aᵀδ = Aᵀ*δ

# state evolution matrix
@time Φ = exp(Matrix(A) * δ);
@time Φᵀ = exp(Matrix(Aᵀ) * δ);

# case with forcing
@time sysnormf = normalize(sysf)
Af = state_matrix(sysnormf);
Afᵀ = copy(transpose(Af));
Afδ = Af*δ
Afᵀδ = Afᵀ*δ;
Φf = exp(Matrix(Af) * δ);
Φfᵀ = exp(Matrix(Afᵀ) * δ);

X0 = Hyperrectangle(low=vcat(zeros(N-1), 1.0, zeros(N)),
                    high=vcat(zeros(N-1), 1.0, zeros(N)))

NSTEPS = 36000
k = 1000
e1, e2 = edirs(k, N)

@time out1K = free_nobloating_krylov(X0, Aᵀδ, e1, NSTEPS; hermitian=false, m=min(30, size(Aᵀδ, 1)), tol=1e-10);
@time out2K = free_nobloating_krylov(X0, Aᵀδ, e2, NSTEPS; hermitian=false, m=min(30, size(Aᵀδ, 1)), tol=1e-10);

In [ ]:
@time intervalos = [ReachSet(Interval(-out2K[k+1], out1K[k+1]),
                             IA.Interval(k*δ, (k+1)*δ)) for k in 0:NSTEPS-1]

fp = Flowpipe(intervalos);

plot(fp, vars=(0, 1), xlab="time", ylab="displacement $k", lab="", lw=0.1)

---

In [ ]:
norm(out1 - out1K)

In [ ]:
norm(out2 - out2K)

In [ ]:
using ExponentialUtilities

# Compute the sequence:
#
# ρ(ℓ, V), ρ(ℓ, Φ V ⊕ V), ρ(ℓ, Φ^2 V ⊕ Φ V ⊕ V), ..,
#
# Method:
#
# out[1] <- ρ(ℓ, V)
# 
# Given that
# ρ(ℓ, Φ V ⊕ V) = ρ(ℓ, Φ V) + ρ(ℓ, V)
#
# Then,
# aux <- ρ(ℓ, Φ V)
#
# out[2] <- aux + out[1]
#
function forced_zerostate_nobloating_krylov(V, Aᵀ, ℓ, NSTEPS;
                                            hermitian=false, m=min(30, size(Aᵀ, 1)), tol=1e-7)

    out = Vector{Float64}(undef, NSTEPS)

    TA, Tb = eltype(Aᵀ), eltype(ℓ)
    T = promote_type(TA, Tb)
    Ks = KrylovSubspace{T, real(T)}(length(ℓ), m)
    arnoldi!(Ks, Aᵀ, ℓ; m=m, ishermitian=hermitian, tol=tol)

    rᵢ = deepcopy(ℓ)

    out[1] = ρ(ℓ, V)

    @inbounds for i in 2:NSTEPS
        expv!(rᵢ, (i-1)*1.0, Ks)
        out[i] = out[i-1] + ρ(rᵢ, V)
    end
    return out
end

In [ ]:
using ExponentialUtilities

# Compute the sequence:
#
# ρ(ℓ, X0), ρ(ℓ, Φ X0 ⊕ V), ρ(ℓ, Φ^2 X0 ⊕ Φ V ⊕ V), ...
#
# Method:
#
# out[1] <- ρ(ℓ, X0) + 0
# 
# out[2] <- ρ(ℓ, Φ X0 ⊕ V) = ρ(ℓ, Φ X0) + ρ(ℓ, V) = ρ(Φᵀ ℓ, X0) + ρ(ℓ, V)
#
# out[3] <- ρ(ℓ, Φ^2 X0 ⊕ Φ V ⊕ V) = ρ((Φᵀ)^2 ℓ, X0) + ρ(Φᵀ ℓ, V) + ρ(ℓ, V)
#
# out[4] <- ρ(ℓ, Φ^3 X0 ⊕ Φ^2 V ⊕ Φ V ⊕ V) = ρ((Φᵀ)^3 ℓ, X0) + ρ((Φᵀ)^2 ℓ, V) + ρ(Φᵀ ℓ, V) + ρ(ℓ, V)
#
function forced_nobloating_krylov_2(X0, V, Aᵀδ, ℓ, NSTEPS;
                                    hermitian=false, m=min(30, size(Aᵀδ, 1)), tol=1e-7)

    out = Vector{Float64}(undef, NSTEPS)
    
    # stores [0, ρ(ℓ, V), ρ(Φᵀ ℓ, V) + ρ(ℓ, V), ρ((Φᵀ)^2 ℓ, V) + ρ(Φᵀ ℓ, V) + ρ(ℓ, V)]
    s = Vector{Float64}(undef, NSTEPS+1)

    TA, Tb = eltype(Aᵀδ), eltype(ℓ)
    T = promote_type(TA, Tb)
    Ks = KrylovSubspace{T, real(T)}(length(ℓ), m)
    arnoldi!(Ks, Aᵀδ, ℓ; m=m, ishermitian=hermitian, tol=tol)
    rᵢ = deepcopy(ℓ)
    rᵢ₋₁ = deepcopy(ℓ)

    @inbounds for i in 1:NSTEPS
        out[i] = ρ(rᵢ, X0) + s[i]
        s[i+1] = s[i] + ρ(rᵢ, V)
        expv!(rᵢ, i*1.0, Ks) # rᵢ <- (Φᵀ)^i ℓ := exp(Aᵀ * δ * i) ℓ
    end
    return out
end

In [ ]:
# definitions
δ = 1e-7;

# case with forcing
N = 1000
sysf = clamped_forced(N=N)
@time sysnormf = normalize(sysf)
Af = state_matrix(sysnormf);
Afᵀ = copy(transpose(Af));
Afδ = Af*δ
Afᵀδ = Afᵀ*δ;
Φf = exp(Matrix(Af) * δ);
Φfᵀ = exp(Matrix(Afᵀ) * δ);

@time M = ReachabilityAnalysis.Φ₁(Matrix(Af), δ, Val(:base));

V = M * sysnormf.U.U.element;
V = Singleton(V);

NSTEPS = 36_000
N = 1000
k = 700
e1, e2 = edirs(k, N)

# see Mechanical Vibrations, Gerardin et al, page 250-251 < Force relates to velocity initial cond
Fext = 10e3
N=1000; E=30e6; A=1; L=200;
L0 = L / N
v0 = Fext / (E * A) * L0
X0 = vcat(vcat(zeros(N-1), v0), zeros(N))  |> Singleton
X0.element[1000]

#@time out1K = forced_nobloating_krylov_2(X0, V, Afᵀδ, e1, NSTEPS; hermitian=false, m=min(30, size(Afᵀδ, 1)), tol=1e-10);
#@time out2K = forced_nobloating_krylov_2(X0, V, Afᵀδ, e2, NSTEPS; hermitian=false, m=min(30, size(Afᵀδ, 1)), tol=1e-10);

@time out1K = free_nobloating_krylov(V, Afᵀδ, e1, NSTEPS)
@time out2K = free_nobloating_krylov(V, Afᵀδ, e2, NSTEPS)

@time intervalos = [ReachSet(Interval(-out2K[k+1], out1K[k+1]), IA.Interval(k*δ, (k+1)*δ)) for k in 0:NSTEPS-1]

fp = Flowpipe(intervalos);
@show numrsets(fp)

plot(fp[1:1:1000], vars=(0, 1), xlab="time", ylab="displacement $k", lab="", lw=0.1)

$$
\rho(d, X) = \max_{x \in X} \langle d, x \rangle
$$

In [ ]:
NSTEPS = 36_000

ℓ = e1
@time out1K = free_nobloating_krylov(X0f, Afᵀδ, ℓ, NSTEPS; hermitian=false, m=min(30, size(Afᵀδ, 1)), tol=1e-10);

ℓ = e2
@time out2K = free_nobloating_krylov(X0f, Afᵀδ, ℓ, NSTEPS; hermitian=false, m=min(30, size(Afᵀδ, 1)), tol=1e-10);

## << OLD >>

In [ ]:
prob = @ivp(sys, x(0) ∈ X0)
k = 700
dirs = edirs(k, N)
# we should go to 0.016
@time solL = solve(prob, tspan=(0.0, 1e-5), alg=LGG09(δ=1e-8, template=dirs));

#k = 3
#plot(solL, vars=(0, k), lw=.2, xlab="time", lab="displacement: $k")
#plot!(solL, vars=(0, 2*k), lw=.2, xlab="time", lab="velocity: $k")

GC.gc()

length(solL)

plot(solL[1:end], vars=(0, k), lw=0.1)

In [ ]:
N = 1000
sys = clamped_ivp(N=N)
δ = 1e-7

# ConstrainedLinearContinuousSystem
@time sysnorm = normalize(sys)
A = state_matrix(sysnorm);
Aᵀ = copy(transpose(A))

@time Φ = exp(Matrix(A) * δ);
@time Φᵀ = exp(Matrix(Aᵀ) * δ);

In [ ]:
X0 = BallInf(ones(2N), 0.1)
e1 = dirs.directions[1]
e2 = dirs.directions[2];

In [ ]:
e1s = SingleEntryVector(k, 2N, 1.0);
e2s = SingleEntryVector(k, 2N, -1.0);

In [ ]:
@btime ρ($e1s, $X0); # 4 ns

In [ ]:
GC.gc()

In [ ]:
ρ(exp(Aᵀ) * ℓ, Ω₀)

In [ ]:

# prob = @ivp(sys, x(0) ∈ X0)
#ρ(ℓ, Ω₀), ρ(exp(Aᵀ) * ℓ, Ω₀), ρ(exp(2Aᵀ) * ℓ, Ω₀)

#=

prob = @ivp(sys, x(0) ∈ X0)
k = 700
dirs = edirs(k, N)
# we should go to 0.016
@time solL = solve(prob, tspan=(0.0, 1e-6), alg=LGG09(δ=1e-8, template=dirs));

=#

#k = 3
#plot(solL, vars=(0, k), lw=.2, xlab="time", lab="displacement: $k")
#plot!(solL, vars=(0, 2*k), lw=.2, xlab="time", lab="velocity: $k")

In [ ]:
idx += 1
A, Aᵀδ, Ω₀, ℓ = heat03(δ=0.02)

# warm-up run
out = Vector{Float64}(undef, 1)
reach_homog_dir_LGG09_expv_pk2!(out, Ω₀, Aᵀδ, sparse(ℓ), 1, recursive, m=94, tol=1e-8, hermitian=true)

out = Vector{Float64}(undef, NSTEPS)
results[model][cases[idx]] = @elapsed reach_homog_dir_LGG09_expv_pk2!(out, Ω₀, Aᵀδ,
                        sparse(ℓ), NSTEPS, recursive, m=94, tol=1e-8, hermitian=true)
max_out = maximum(out)
max_temp[cases[idx]] = max_out
property = max_out ∈ Tmax[3] .. Tmax[3] + Δ
push!(validation, Int(property))

out = nothing
GC.gc()

In [ ]:
N = 1000
sys = clamped_ivp(N=N)
X0 = BallInf(ones(2N), 0.1)
prob = @ivp(sys, x(0) ∈ X0)
k = 700
dirs = edirs(k, N)
# we should go to 0.016
@time solL = solve(prob, tspan=(0.0, 1e-6), alg=LGG09(δ=1e-8, template=dirs));